In [1]:
import ee
import numpy as np
import pandas as pd

Conectarse a GEE

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-apr160')
print(ee.String('Hello from the Earth Engine servers!').getInfo())


Successfully saved authorization token.
Hello from the Earth Engine servers!


Definir variables   

In [4]:
year = 2020

In [5]:
df_data = pd.read_csv(f'data/{year}.csv',encoding = "ISO-8859-1")
df_data = df_data[['Latitud','Longitud']].drop_duplicates().reset_index(drop=True)
df_data.to_csv(f'data/{year}-unique-points.csv')

Definicion de funciones

In [7]:
def add_date(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

def save_coordinate(Sentinel_data,row, feature):
    features = []
    poi_geometry = ee.Geometry.Point([row['Longitud'], row['Latitud']])
    poi_properties = dict(row)
    poi_feature = ee.Feature(poi_geometry, poi_properties)
    features.append(poi_feature)
    ee_fc = ee.FeatureCollection(features) 


    def rasterExtraction(image):
        feature = image.sampleRegions(
            collection = ee_fc, # feature collection here
            scale = 10 # Cell size of raster
        )
        return feature
    
    results = Sentinel_data.filterBounds(ee_fc) \
            .select(feature).map(add_date) \
            .map(rasterExtraction) \
            .flatten()

    # You can modify this for better optimization
    column_df = list(df_data.columns)
    column_df.extend([feature, 'date'])
    
    nested_list = results.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
    data = nested_list.getInfo()
    df_final = pd.DataFrame(data,columns=column_df)
    
    df_final.to_csv('coordinates/'+feature+'/' + str(row['Longitud']) + '_' + str(row['Latitud']) + '.csv')
   

def get_data(source,from_date,to_date,feature):
    Sentinel_data = ee.ImageCollection(source) \
    .filterDate(from_date,to_date) \
    .map(add_date)

    for index, row in df_data.iterrows():
        save_coordinate(Sentinel_data,row,feature)
        print('.',end='')

In [24]:
get_data('NOAA/CDR/AVHRR/LAI_FAPAR/V5',"2021-09-20","2022-12-31",'LAI')